In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/IdeaProjects/neural-network-convolution-autodiff-libs/MY-MLP-IMPL`


In [2]:
using JLD2
X_train = load("../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/imdb_dataset_prepared.jld2", "y_test")
nothing

In [9]:
using MYMLP, Printf, Statistics

dataset = DataLoader((X_train, y_train), batchsize=64, shuffle=true)

model = Chain(
    Dense(size(X_train, 1), 32, relu),
    Dense(32, 1, sigmoid)
)

loss(m, x, y) = binarycrossentropy(m(x), y)
accuracy(m, x, y) =  mean((m(x).output .> 0.5) .== (y .> 0.5))

opt = setup(Adam(), model)
epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = gradient!(model) do m
                l = loss(m, x, y)
                total_loss += l.output
                total_acc += accuracy(m, x, y)
                return l
            end
            update!(opt, model, grads[1])
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test).output
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (3.25s) 	Train: (l: 0.63, a: 0.84) 	Test: (l: 0.58, a: 0.85)
Epoch: 2 (2.90s) 	Train: (l: 0.44, a: 0.92) 	Test: (l: 0.44, a: 0.86)
Epoch: 3 (2.91s) 	Train: (l: 0.29, a: 0.94) 	Test: (l: 0.37, a: 0.87)
Epoch: 4 (2.93s) 	Train: (l: 0.20, a: 0.96) 	Test: (l: 0.34, a: 0.87)
Epoch: 5 (2.76s) 	Train: (l: 0.14, a: 0.97) 	Test: (l: 0.32, a: 0.88)
